# Tutorial: Build and Host Machine Learning Demos with Gradio ⚡ & Hugging Face 🤗

**Learning goals:** The goal of this tutorial is to learn How To

1. Build a quick demo for your machine learning model in Python using the `gradio` library
2. Host the demos for free with Hugging Face Spaces
3. Add your demo to your Hugging Face org

## Why Demos?

**Demos** of machine learning models are an increasingly important part of machine learning _courses_ and _conferences_. Demos allow:

* model developers to easily **present** their work to a wide audience
* increase **reproducibility** of machine learning research
* diverse users to more easily **identify and debug** failure points of models


As a quick example of what we would like to build, check out the [Keras Org on Hugging Face](https://huggingface.co/keras-io), which includes a description card and a collection of Models and Spaces built by Keras community. Any Space can be opened in your browser and you can use the model immediately, as shown here:

![](https://i.ibb.co/7y6DGjB/ezgif-5-cc52b7e590.gif)




## 1. Build Quick ML Demos in Python Using the Gradio Library

`gradio` is a handy Python library that lets you build web demos simply by specifying the list of input and output **components** expected by your machine learning model.

What do I mean by input and output components? Gradio comes with a bunch of predefined components for different kinds of machine learning models. Here are some examples:

* For an **image classifier**, the expected input type is an `Image` and the output type is a `Label`.
* For a **speech recognition model**, the expected input component is an `Microphone` (which lets users record from the browser) or `Audio` (which lets users drag-and-drop audio files), while the output type is `Text`.
* For a **question answering model**, we expect **2 inputs**: [`Text`, `Text`], one textbox for the paragraph and one for the question, and the output type is a single `Text` corresponding to the answer.

You get the idea... (for all of the supported components, [see the docs](https://gradio.app/docs/))

In addition to the input and output types, Gradio expects a third parameter, which is the prediction function itself. This parameter can be ***any* regular Python function** that takes in parameter(s) corresponding to the input component(s) and returns value(s) corresponding to the output component(s)

Enough words. Let's see some code!

In [ ]:
# First, install Gradio
!pip install --quiet gradio

In [ ]:
import numpy as np

def sepia(image):
    sepia_filter = np.array(
        [[0.393, 0.769, 0.189],
         [0.349, 0.686, 0.168],
         [0.272, 0.534, 0.131]]
    )
    sepia_img = image.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

In [ ]:
import gradio as gr

# Write 1 line of Python to create a simple GUI
gr.Interface(fn=sepia, inputs="image", outputs="image").launch();

Running the code above should produce a simple GUI inside this notebook allowing you to type example inputs and see the output returned by your function.

Notice that we define an `Interface` using the 3 ingredients mentioned earlier:
* A function
* Input component(s)
* Output component(s)

This is a simple example for images, but the same principle holds true for any other kind of data type. For example, here is an interface that generates a musical tone when provided a few different parameters (the specific code inside `generate_tone()` is not important for the purpose of this tutorial):

In [ ]:
import numpy as np
import gradio as gr

def generate_tone(note, octave, duration):
    sampling_rate = 48000
    a4_freq, tones_from_a4 = 440, 12 * (octave - 4) + (note - 9)
    frequency = a4_freq * 2 ** (tones_from_a4 / 12)
    audio = np.linspace(0, int(duration), int(duration) * sampling_rate)
    audio = (20000 * np.sin(audio * (2 * np.pi * frequency))).astype(np.int16)
    return sampling_rate, audio

gr.Interface(
    generate_tone,
    [
        gr.Dropdown(["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"], type="index"),
        gr.Slider(4, 6, step=1),
        gr.Number(value=1, label="Duration in seconds"),
    ],
    "audio",
    title="Generate a Musical Tone!"
).launch()

**Challenge #1**: build a Gradio demo that takes in an image and returns the same image *flipped upside down* in less than 10 lines of Python code.

There are a lot more examples you can try in Gradio's [getting started page](https://gradio.app/getting_started/), which cover additional features such as:
* Adding example inputs
* Adding _state_ (e.g. for chatbots)
* Sharing demos easily using one parameter called `share` (<-- this is pretty cool 😎)

It is especially easy to demo a `transformers` model from Hugging Face's Model Hub, using the special `gr.Interface.load` method.

Let's try a text-to-speech model built by Facebook:

In [ ]:
import gradio as gr

gr.Interface.load("huggingface/facebook/fastspeech2-en-ljspeech").launch();

Here is the code to build a demo for [GPT-J](https://huggingface.co/EleutherAI/gpt-j-6B), a large language model & add a couple of examples inputs:

In [ ]:
import gradio as gr

examples = [["The Moon's orbit around Earth has"], ["There once was a pineapple"]]

gr.Interface.load("huggingface/EleutherAI/gpt-j-6B", examples=examples).launch();

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation")

In [ ]:
generator("Hello world, my name is")[0]['generated_text']

In [ ]:
import gradio as gr

def generate_text(text):
  return generator(text)[0]['generated_text']


# Write 1 line of Python to create a simple GUI
gr.Interface(fn=generate_text, inputs="text", outputs="text").launch();

# Challenge! Create your own Space

**Challenge #2**: Go to the [Hugging Face Model Hub](https://huggingface.co/models), and pick a model that performs one of the other tasks supported in the `transformers` library (other than the two you just saw: text generation or text-to-speech). Create a Gradio demo for that model using `gr.Interface.load`

OR, if you're feeling adventurous... create a Gradio demo using the `pipeline` function from `transformers`...

In [ ]:
# Write your code in this cell!

# 1. Find a model on https://hf.co/models?library=transformers&sort=downloads

# 2. Decide on what inputs and outputs you'll need

# 3. Create a Gradio app for the model! (How will you load and use the model?)

import gradio as gr

def fill_image(image):
  return image["image"]

gr.Interface(
    fn=fill_image,
    inputs=[
        gr.Image(tool="sketch")
    ],
    outputs="image"
).launch()

## Host the Demo (for free) on Hugging Face Spaces

Once you made a Gradio demo, you can host it permanently on Hugging Spaces very easily:

Here are the steps to that (shown in the GIF below):

A. First, create a Hugging Face account if you do not already have one, by visiting https://huggingface.co/ and clicking "Sign Up"

B. Once you are logged in, click on your profile picture and then click on "New Space" underneath it to get to this page: https://huggingface.co/new-space

C. Give your Space a name and a license. Select "Gradio" as the Space SDK, and then choose "Public" if you are fine with everyone accessing your Space and the underlying code

D. Then you will find a page that provides you instructions on how to upload your files into the Git repository for that Space. You may also need to add a `requirements.txt` file to specify any Python package dependencies.

E. Once you have pushed your files, that's it! Spaces will automatically build your Gradio demo allowing you to share it with anyone, anywhere!

![GIF](https://huggingface.co/blog/assets/28_gradio-spaces/spaces-demo-finalized.gif)



